### 코드 수정
### Dataloader 문제 수정
### epoch별 검증 1-MAE 출력

In [ ]:
import os
import logging
os.environ["CUDA_VISIBLE_DEVICES"] = "2"  # only use GPU 2
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s]::%(module)s::%(levelname)s::%(message)s')
streamHandler = logging.StreamHandler()
streamHandler.setFormatter(formatter)
fileHandler = logging.FileHandler('./LOG/basein-tuning.log')
fileHandler.setFormatter(formatter)
logger.addHandler(streamHandler)
logger.addHandler(fileHandler)

In [2]:
import pandas as pd
logger.debug('=============Mbart-large-50 instruction tuning Strat!=============')

df = pd.read_csv("/home/oem/qx/in_tuning/KETI_nopreprocessed_labeled_dropdup+seperate_ver0.3.csv")

[2025-03-30 23:45:46,820]::3848060699::DEBUG::=============Mbart-large-50 instruction tuning Strat!=============


In [3]:
# 데이터 크기 확인
print(df.shape)

(11307, 10)


In [7]:
df = df[['transcription', 'OCEAN']]
print(df.head())

                   transcription  \
0  저희가 지금 봐야 되는 게 설치 장소랑 뭘 팔지네요.   
1            그러니까 계절이 10월이니까 쌀쌀한   
2                          좀 쌀쌀한   
3         날씨 그럼 뭐 각자 좀 생각을 해볼까요?   
4      아니면 그냥 저희끼리 얘기를 하는 게 편하신지   

                                               OCEAN  
0  [0.39583333333333337, 0.7708333333333333, 0.72...  
1  [0.39583333333333337, 0.7708333333333333, 0.72...  
2  [0.7708333333333333, 0.5416666666666666, 0.770...  
3  [0.39583333333333337, 0.7708333333333333, 0.72...  
4  [0.39583333333333337, 0.7708333333333333, 0.72...  


In [8]:
import ast 

# OCEAN 컬럼을 리스트로 변환 후 소숫점 4자리로 반올림
def round_ocean_values(ocean_string):
    ocean_list = ast.literal_eval(ocean_string)  # 문자열을 리스트로 변환
    return [round(value, 4) for value in ocean_list]  # 각 값을 소숫점 4자리로 반올림

In [9]:
# OCEAN 값 처리
df['OCEAN'] = df['OCEAN'].apply(round_ocean_values)

# 확인
print(df[['transcription', 'OCEAN']].head())

                   transcription                                     OCEAN
0  저희가 지금 봐야 되는 게 설치 장소랑 뭘 팔지네요.  [0.3958, 0.7708, 0.7292, 0.8333, 0.4375]
1            그러니까 계절이 10월이니까 쌀쌀한  [0.3958, 0.7708, 0.7292, 0.8333, 0.4375]
2                          좀 쌀쌀한  [0.7708, 0.5417, 0.7708, 0.8125, 0.3542]
3         날씨 그럼 뭐 각자 좀 생각을 해볼까요?  [0.3958, 0.7708, 0.7292, 0.8333, 0.4375]
4      아니면 그냥 저희끼리 얘기를 하는 게 편하신지  [0.3958, 0.7708, 0.7292, 0.8333, 0.4375]


In [ ]:
import torch
instruction = "주어진 대화를 보고, OCEAN 다섯 가지 성격 특성을 0에서 1 사이의 점수로 예측하세요."

# Instruction + Transcription 결합
df['input_text'] = df['transcription'].apply(
    lambda x: f"### Instruction:\n{instruction}\n\n### Input:\n{x}\n\n### Response:"
)

# 필요 없는 컬럼 제거
df = df[['input_text', 'OCEAN']]

print(df.head())  # 데이터 확인

                                          input_text  \
0  ### Instruction:\n주어진 대화를 보고, OCEAN 다섯 가지 성격 특...   
1  ### Instruction:\n주어진 대화를 보고, OCEAN 다섯 가지 성격 특...   
2  ### Instruction:\n주어진 대화를 보고, OCEAN 다섯 가지 성격 특...   
3  ### Instruction:\n주어진 대화를 보고, OCEAN 다섯 가지 성격 특...   
4  ### Instruction:\n주어진 대화를 보고, OCEAN 다섯 가지 성격 특...   

                                      OCEAN  
0  [0.3958, 0.7708, 0.7292, 0.8333, 0.4375]  
1  [0.3958, 0.7708, 0.7292, 0.8333, 0.4375]  
2  [0.7708, 0.5417, 0.7708, 0.8125, 0.3542]  
3  [0.3958, 0.7708, 0.7292, 0.8333, 0.4375]  
4  [0.3958, 0.7708, 0.7292, 0.8333, 0.4375]  


In [11]:
from datasets import Dataset

# Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df)
dataset = dataset.shuffle(seed=42)  # 데이터 섞기
dataset = dataset.train_test_split(test_size=0.2)  # 80% 학습, 20% 검증

# 데이터 확인
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['input_text', 'OCEAN'],
        num_rows: 9045
    })
    test: Dataset({
        features: ['input_text', 'OCEAN'],
        num_rows: 2262
    })
})


In [12]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# 选择一个 Seq2Seq 的 XLM-R 类似模型（XLM-R 本身没有 Seq2Seq 版本）
model_name = "/home/oem/qx/test_string_and_float/best_model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")
tokenizer.src_lang = "ko_KR"
tokenizer.tgt_lang = "ko_KR"

2025-03-30 23:45:50.905228: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-30 23:45:50.908903: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-30 23:45:50.916105: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743345950.927980 3629859 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743345950.931517 3629859 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been regist

In [ ]:
import torch
# 토크나이징 함수 정의
def preprocess_data(examples):
    inputs = tokenizer(examples["input_text"], padding="max_length", truncation=True, max_length=256)
    # OCEAN 값을 문자열로 변환하여 Target 생성
    targets = [" ".join(map(str, ocean)) for ocean in examples["OCEAN"]]
    labels = tokenizer(targets, padding="max_length", truncation=True, max_length=256)
    
    # 반환할 딕셔너리 생성input_ids,attention_mask
    inputs["labels"] = labels["input_ids"]
    return inputs

In [ ]:
# 데이터셋 전처리 적용
train_dataset = dataset["train"].map(preprocess_data, batched=True)
eval_dataset = dataset["test"].map(preprocess_data, batched=True)
# 학습용 데이터 확인
print(eval_dataset[0])
print(eval_dataset)

Map:   0%|          | 0/9045 [00:00<?, ? examples/s]

Map:   0%|          | 0/2262 [00:00<?, ? examples/s]

{'input_text': '### Instruction:\n주어진 대화를 보고, OCEAN 다섯 가지 성격 특성을 0에서 1 사이의 점수로 예측하세요.\n\n### Input:\n동쪽 괜찮을 것 같은데\n\n### Response:', 'OCEAN': [0.4375, 0.7083, 0.5625, 0.3958, 0.4167], 'input_ids': [250014, 6, 187284, 72022, 10763, 12, 5837, 2211, 4033, 123541, 688, 30861, 4, 91888, 105621, 6685, 152915, 44276, 182686, 133954, 413, 757, 1180, 106, 62657, 367, 202031, 1083, 211048, 44476, 5, 6, 187284, 360, 7077, 12, 8354, 39048, 142324, 413, 6668, 13928, 16404, 6, 187284, 92748, 12, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [16]:
import os

os.environ["NCCL_P2P_DISABLE"] = "1"
os.environ["NCCL_IB_DISABLE"] = "1"

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def my_collate_fn(batch):
    input_texts = [item['input_text'] for item in batch]
    labels = torch.stack([torch.tensor(item['OCEAN']) for item in batch])
    return {"input_text": input_texts, "OCEAN": labels}
from torch.utils.data import DataLoader
eval_dataloader = DataLoader(eval_dataset, batch_size=8,collate_fn=my_collate_fn)

In [18]:
from tqdm import tqdm
def evaluate_ocean(model, tokenizer, eval_dataloader, device):
    import re
    from sklearn.metrics import mean_absolute_error

    model.eval()
    all_test_predictions = []
    all_test_labels = []
    fail_count = 0

    for batch in tqdm(eval_dataloader):
        test_input_texts = batch['input_text']
        test_labels = batch["OCEAN"]
        inputs = tokenizer(test_input_texts, return_tensors="pt", padding=True, truncation=True).to(device)
        outputs = model.generate(
            **inputs,
            max_length=64,
            temperature=1.2,
            top_p=0.9,
            repetition_penalty=1.2
        )

        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)

        for pred_str, label in zip(decoded_preds, test_labels):
            pred_str = re.sub(r"[^\d\.\,\s]", "", pred_str)
            try:
                prediction = list(map(float, re.split(r"[,\s]+", pred_str.strip())))
                # print(prediction)
                if len(prediction) == 5 and len(label) == 5:
                    all_test_predictions.extend(prediction)
                    all_test_labels.extend(label.tolist())
                else:
                    fail_count += 1
            except:
                fail_count += 1

    print(f"⚠️ Skipped {fail_count} invalid predictions.")
    model.train()

    mae_score = mean_absolute_error(all_test_labels, all_test_predictions)
    one_minus_mae = 1 - mae_score
    logger.debug(f"\n🧪 Custom Evaluation — 1 - MAE: {one_minus_mae:.4f}")

    return one_minus_mae

In [ ]:
def my_train_collate_fn(batch):
    input_ids = torch.stack([torch.tensor(item['input_ids']) for item in batch])
    attention_mask = torch.stack([torch.tensor(item['attention_mask']) for item in batch])
    labels = torch.stack([torch.tensor(item['labels']) for item in batch])  
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [20]:
import shutil
from transformers import TrainerCallback, TrainerControl, TrainerState

class OceanEvalCallback(TrainerCallback):
    def __init__(self, eval_dataloader, tokenizer, device, output_dir="best_model"):
        self.eval_dataloader = eval_dataloader
        self.tokenizer = tokenizer
        self.device = device
        self.output_dir = output_dir
        self.best_score = -float("inf")

    def on_epoch_end(self, args, state: TrainerState, control: TrainerControl, **kwargs):
        model = kwargs['model']
        print("\n🔍 Running custom evaluation at end of epoch...")

        score = evaluate_ocean(model, self.tokenizer, self.eval_dataloader, self.device)

        if score > self.best_score:
            self.best_score = score
            logger.debug(f"💾 New best model found with score {score:.4f} — saving to {self.output_dir}")

            # 清空旧模型文件夹（如果存在）
            if os.path.exists(self.output_dir):
                shutil.rmtree(self.output_dir)

            # 保存当前最优模型
            model.save_pretrained(self.output_dir)
            self.tokenizer.save_pretrained(self.output_dir)

In [ ]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

# 학습 설정
training_args = TrainingArguments(
    output_dir="./baseresults",
    num_train_epochs=50,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="no",  
    save_strategy="no",
    logging_strategy="steps",
    logging_steps=1000,
    save_total_limit=2,
    load_best_model_at_end=False
    # fp16=True
)

callback = OceanEvalCallback(
    eval_dataloader=eval_dataloader,
    tokenizer=tokenizer,
    device=device,
    output_dir="./basebest_model"
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    callbacks=[callback],
    data_collator=my_train_collate_fn
)
print(train_dataset)
# 모델 학습 시작
trainer.train()
savefile = "./basembart-large-50-float/epoch100"
trainer.save_model(savefile)
tokenizer.save_pretrained(savefile)

/home/oem/anaconda3/envs/qx/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_3629859/2487976153.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Dataset({
    features: ['input_text', 'OCEAN', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 9045
})


Step,Training Loss
1000,0.017000
2000,0.017800
3000,0.016100
4000,0.033500
5000,0.016300
6000,0.015800
7000,0.015900
8000,0.015200
9000,0.014900
10000,0.014900



🔍 Running custom evaluation at end of epoch...


  0%|          | 0/283 [00:00<?, ?it/s]/home/oem/anaconda3/envs/qx/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/oem/anaconda3/envs/qx/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 283/283 [01:05<00:00,  4.34it/s]
[2025-03-30 23:52:26,929]::3631506463::DEBUG::
🧪 Custom Evaluation — 1 - MAE: 0.8311
[2025-03-30 23:52:26,930]::4125021081::DEBUG::💾 New best model found with score 0.8311 — saving to ./basebest_model


...

  0%|          | 0/283 [00:00<?, ?it/s]/home/oem/anaconda3/envs/qx/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/oem/anaconda3/envs/qx/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 283/283 [01:11<00:00,  3.96it/s]
[2025-03-31 05:07:35,058]::3631506463::DEBUG::
🧪 Custom Evaluation — 1 - MAE: 0.3024


⚠️ Skipped 1 invalid predictions.

🔍 Running custom evaluation at end of epoch...


  0%|          | 0/283 [00:00<?, ?it/s]/home/oem/anaconda3/envs/qx/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/oem/anaconda3/envs/qx/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 283/283 [01:12<00:00,  3.92it/s]
[2025-03-31 05:14:09,803]::3631506463::DEBUG::
🧪 Custom Evaluation — 1 - MAE: -0.2329


⚠️ Skipped 1 invalid predictions.


('./basembart-large-50-float/epoch100/tokenizer_config.json',
 './basembart-large-50-float/epoch100/special_tokens_map.json',
 './basembart-large-50-float/epoch100/sentencepiece.bpe.model',
 './basembart-large-50-float/epoch100/added_tokens.json',
 './basembart-large-50-float/epoch100/tokenizer.json')